*Note: This notebook likely will not run correctly through Jupyter. I originally began this project in Deepnote, but ran into performance issues with the free licence so moved to my own computer and Jupyter after Checkpoint 3.*

## Setup

In [ ]:
#Setup kaggle API
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Import datasets from kaggle using the kaggle API
#!kaggle datasets download zaheenhamidani/ultimate-spotify-tracks-db
#!kaggle datasets download jfreyberg/spotify-chart-data
!kaggle datasets download yamaerenay/spotify-dataset-19212020-600k-tracks

# Unzip datasets
#!unzip ultimate-spotify-tracks-db.zip --yes
#!unzip spotify-chart-data.zip --yes
!unzip spotify-dataset-19212020-600k-tracks.zip --yes

In [ ]:
!pip install matplotlib
!pip install scikit-learn

In [1]:
# Setup Spotify API
# Setup Spotipy
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

with open('credentials.json') as f:
    data = json.load(f)
    SPOTIPY_CLIENT_ID = data['SPOTIPY_CLIENT_ID']
    SPOTIPY_CLIENT_SECRET = data['SPOTIPY_CLIENT_SECRET']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET))

In [2]:
from colorama import Fore, Back, Style

from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np
import re 
import sys
import itertools
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("max_rows", None)

In [ ]:
import csv, sqlite3

con = sqlite3.connect("spotify.db")
cur = con.cursor()

In [ ]:
%load_ext sql

In [ ]:
%sql sqlite:///spotify.db

## The plan <a name="the-plan"></a>

![alt text](Spotify-plan.png "Spotify plan")

## The problem

As is clear from the plan, I had wanted to combine and process multiple datasets to create a larger dataset from which to create the recommendation system, for better accuracy. As a safety feature to keep Spotify running smoothly, there is a rate limit on queries, which I discovered when trying to run 20'000 queries pulling artist genres and release years. In light of this, I'll continue by using only one dataset, the [Spotify Dataset 1921-2020, 600k+ Tracks]("https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks?select=tracks.csv"), which already contains all of the features I want to analyse.

In [ ]:
df_artists = pd.read_csv('artists.csv')
#df_artists.to_sql("ARTISTS", con, if_exists='replace', index=False, method="multi")
df_artists.head()

In [ ]:
df_tracks = pd.read_csv('tracks.csv')
#df_tracks.to_sql('TRACKS', con, if_exists='replace', index=False, method="multi")
df_tracks.head()

By using SQL to look at the dataset, it's like there are two tables: `df_artists` and `df_tracks`. They are linked by artist name and id. 

## Feature Engineering

Initially, what I wanted to do was to keep only the top artist of any track for simplicity, as some tracks have more than one artist. However, I discovered the pandas `.explode()` method which will separate the artist list into separate rows. The first issue with doing this however is that the list of artists is not really a list, but a string that looks like a list, so I need to remedy this first.

In [ ]:
df_tracks['artists_upd_v1'] = df_tracks['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))

In [ ]:
df_tracks['artists'].values[0]

In [ ]:
df_tracks['artists_upd_v1'].values[0][0]

This is perfect, but I need to check if it applied to the whole dataset correctly.

In [ ]:
df_tracks[df_tracks['artists_upd_v1'].apply(lambda x: not x)].head(5)

It did not. Some artists with apostrophes in there names screwed up.

In [ ]:
df_tracks['artists_upd_v2'] = df_tracks['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
df_tracks['artists_upd'] = np.where(df_tracks['artists_upd_v1'].apply(lambda x: not x), df_tracks['artists_upd_v2'], df_tracks['artists_upd_v1'] )


In [ ]:
df_tracks.head(100)

Now I can `.explode()` the updated artists column. This will result in rows with duplicate `id`'s since the exploded rows are identical in all but `artists_upd`. This is fine. What is less fine is this scenario.

Say a song has two artists, A and B. The song is likely published on both A and B's Spotify page, so will already have the same `id`, and now we've exploded the list, the track will appear multiple times. 

I'll create a new feature, `artist_upd|song` which concatenates the artist and song names to help spot these duplicates. 

In [ ]:
df_tracks = df_tracks.explode('artists_upd')

In [ ]:
df_tracks['artist_upd|song'] = df_tracks['artists_upd'] + "|" + df_tracks['name']

In [ ]:
df_tracks.drop_duplicates('artist_upd|song', inplace=True)

In [ ]:
df_tracks.shape

In [ ]:
#df_1 = _deepnote_execute_sql('select * from df_tracks limit 10', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
#df_1

I want the song duration to be measured in seconds instead of milliseconds, so I'll change that. At the same time, I'll also truncate all release dates to just contain the year for consistency. I had originally planned to drop the `id` column but I came to realise it could be useful in future, for checking duplicates and for more interesting purposes that I will get too later.

In [ ]:
# Create duration_s column and drop duration_ms
df_tracks['duration_s'] = df_tracks['duration_ms']/1000
df_tracks.drop(['duration_ms'], axis=1, inplace=True)

# Create year and month column and drop release date
df_tracks['year'] = df_tracks['release_date'].str[:4]
df_tracks.drop(['release_date'], axis=1, inplace=True)

In [ ]:
df_1 = _deepnote_execute_sql('select * from df_tracks limit 10', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

Due to the time this notebook takes to run, I will periodically export `csv` files that I can later read in should the host machine shut down for any reason.

### Checkpoint 1

In [ ]:
df_tracks.to_csv('tracks-1.csv', index=False)

The `key` and `mode` columns are two parts of the same metric, notably spokes on the circle of fifths. Whilst I could keep them in the numerical form they are in, I want to combine them into a single column with object values such as `C`, `Db`, `Bbm` etc. There should be 24 unique values. I will then one hot encode these for the purpose of the model.

In [ ]:
df_tracks = pd.read_csv('tracks-1.csv')

In [ ]:
# Recode mode column to contain suffix for new key encoding. No suffix for major, m for minor
df_tracks['mode'].mask(df_tracks['mode'] == 1, 'Major', inplace=True) 
df_tracks['mode'].mask(df_tracks['mode'] == 0, 'Minor', inplace=True) 

In [ ]:
def key_encode(x):
    '''
    Recode key column to letter notation 
    '''
    if x == 0:
        return 'C'
    elif x == 1:
        return 'Db'
    elif x == 2:
        return 'D'
    elif x == 3:
        return 'Eb'
    elif x == 4:
        return 'E'
    elif x == 5:
        return 'F'
    elif x == 6:
        return 'Gb'
    elif x == 7:
        return 'G'
    elif x == 8:
        return 'Ab'
    elif x == 9:
        return 'A'
    elif x == 10:
        return 'Bb'
    elif x == 11:
        return 'B'
    else:
        return 'Bbm'

In [ ]:
# Change numerical key to standard pitch class notation
df_tracks['key_letter'] = df_tracks['key'].apply(key_encode)
df_tracks.drop(['key'], axis=1, inplace=True)

In [ ]:
df_1 = _deepnote_execute_sql('select * from df_tracks limit 10', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

Next I want to bucket popularity into 10 levels between 0 and 100, as the level of granularity it comes in as standard is unneccesary. If I like popular music, how much difference does a song scoring 87 and one scoring 89 really have?

In [ ]:
# Create 10 evenly spaced points betwee 0 and 101 (since digitize is lower<=x<upper)
points = np.linspace(0, 101, 11)

df_tracks['popularity'] = np.digitize(df_tracks['popularity'], points)

In [ ]:
print(df_tracks['popularity'].value_counts())

In [ ]:
df_1 = _deepnote_execute_sql('select * from df_tracks limit 10', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

Next I'll finally merge the two tables together, so that I can use the `genres` data from `artists.csv` in my `df_tracks` data. I'm using an inner join so every artist has a genre list assigned, even if that list is [].

In [ ]:
df_tracks = _deepnote_execute_sql('select df_tracks.*, df_artists.genres as genre_list\nfrom df_tracks\ninner join df_artists on df_tracks.artists_upd = df_artists.name', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_tracks

### Checkpoint 1.5

In [ ]:
# Checkpoint added later for genre cleanup
df_tracks.to_csv('tracks-1.5.csv', index=False)

In [ ]:
df_tracks = pd.read_csv('tracks-1.5.csv')

In [ ]:
# TF-IDF cannot handle null/missing values. If there was any, I would replace them with [], but as you
# can see, I don't need to do that.
print(len(df_tracks[df_tracks['genre_list'] == None]))

In [ ]:
# As with the artists, the 'genre_list' column is not actually a Pyton list, but a string that looks like one. 
# I need to convert it to a proper list.
#df_tracks['artists_upd_v1'] = df_tracks['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
#df_tracks['artists_upd_v2'] = df_tracks['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
#df_tracks['artists_upd'] = np.where(df_tracks['artists_upd_v1'].apply(lambda x: not x), df_tracks['artists_upd_v2'], df_tracks['artists_upd_v1'] )

# THIS CELL HAS BEEN MOVED LATER ON DUE TO WEIRD ERRORS WHEN THE DEEPNOTE MACHINE RUNS OUT OF RAM
# POOR PERSON ISSUE 
#df_tracks['genres_upd_v1'] = df_tracks['genre_list'].apply(lambda x: re.findall(r"'([^']*)'", x))
#df_tracks['genres_upd_v2'] = df_tracks['genre_list'].apply(lambda x: re.findall('\"(.*?)\"',x))
#df_tracks['genres_upd'] = np.where(df_tracks['genres_upd_v1'].apply(lambda x: not x), df_tracks['genres_upd_v2'], df_tracks['genres_upd_v1'] )

The last things to do before one-hot-encoding are to create some combined columns like mode|key for the model to use.

In [ ]:
df_tracks['mode|key'] = df_tracks['mode'] + "|" + df_tracks['key_letter']

In [ ]:
df_1 = _deepnote_execute_sql('select * from df_tracks limit 10', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

### Checkpoint 2

In [ ]:
# Another checkpoint saving to a csv
df_tracks.to_csv('tracks-2.csv', index=False)

In [ ]:
df_tracks = pd.read_csv('tracks-2.csv')

The final FE task will be to one-hot-encode a large portion of the data. Before I do this, I want to have one last check for null and duplicate values, and to check datatypes.

In [ ]:
print("There are",df_tracks.duplicated(['artist_upd|song']).sum(), "duplicate values")
print(df_tracks.isnull().sum())

In [ ]:
# I don't understand how there are artist_upd|song duplicates when I explicitly got rid of them earlier.
# Regardless I'll get rid of them (again)
df_tracks.drop_duplicates(subset=['artist_upd|song'], inplace=True)

# The null values in month represent entries for which the Spotify API only returned the year.
# I could drop these entirely, but as that is a sizable chunk of the data, I'll keep them as they are.

In [ ]:
df_tracks.dtypes

In [ ]:
df_tracks[['popularity', 'explicit']] = df_tracks[['popularity', 'explicit']].astype(float)

In [ ]:
df_1 = _deepnote_execute_sql('select * from df_tracks limit 100', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_1

Now for one-hot-encoding.

In [4]:
def ohe_prep(df, col, new_name):
    '''
    Create one-hot-encoded features of a specific column

    Parameters:
        df (pd.DataFrame): dataframe
        col (str): column name
        new_name (str): new column name

    Returns:
        ohe: one-hot-encoded dataframe
    '''
    
    ohe = pd.get_dummies(df[col])
    feature_names = ohe.columns
    ohe.columns = [new_name + "|" + str(x) for x in feature_names]
    ohe.reset_index(drop=True, inplace=True)
    return ohe

In [5]:
def create_feature_set(df, float_cols):
    '''
    Process df to create final feature set which will be used for recommendations. 

    Parameters:
        df (pd.DataFrame): dataframe
        float_cols (list(str)): list of float columns

    Returns:
        final: final feature set
    '''

    # Tfidf genre lists
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['genres_upd'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = ['genre' + '|' + x for x in tfidf.get_feature_names_out()]
    genre_df.reset_index(drop=True, inplace=True)

    # Ohe with personal weightings
    year_ohe = ohe_prep(df, 'year', 'year') * 0.5 
    popularity_ohe = ohe_prep(df, 'popularity', 'pop') * 0.15
    ts_ohe = ohe_prep(df, 'time_signature', 't_sig') * 0.15

    # Scale float columns
    floats = df[float_cols].reset_index(drop=True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns=floats.columns) #* 0.2

    # Concatenate all features
    final = pd.concat([genre_df, floats_scaled, popularity_ohe, year_ohe, ts_ohe], axis=1)

    # Add song ID
    final['id'] = df['id'].values

    return final

### Checkpoint 3

In [ ]:
# Another checkpoint save
#df_tracks.to_csv('tracks-3.csv', index=False)

In [6]:
df_tracks = pd.read_csv('tracks-3.csv')

In [7]:
df_tracks.head(50)

,id,name,popularity,explicit,artists,id_artists,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artists_upd_v1,artists_upd_v2,artists_upd,artist_upd|song,duration_s,year,key_letter,genre_list,mode|key
0,36OG5Ep6IPA4Mj6TxoTpcw,Jab Ham Chale To Saya Bhi Apna - Dialogues,1.0,0.0,['Guru Dutt'],['7DLDYPMRJ0NFPcBTlg04DE'],0.439,0.3110,-14.167,Major,0.3270,0.96000,0.243000,0.2110,0.39200,77.238,4,['Guru Dutt'],[],Guru Dutt,Guru Dutt|Jab Ham Chale To Saya Bhi Apna - Dia...,31.013,1957,Db,[],Major|Db
1,4TDSHnJFIzcLkOEc8hoRlm,Rockin' and Rollin' in Hawaii,1.0,0.0,['The De Castro Sisters'],['1OCPhFtvkZDLUJJkrJfD2G'],0.436,0.5600,-9.263,Major,0.0468,0.67700,0.000000,0.0829,0.90500,147.300,4,['The De Castro Sisters'],[],The De Castro Sisters,The De Castro Sisters|Rockin' and Rollin' in H...,126.118,1955,Ab,[],Major|Ab
2,22aStKnRzVJsTIECFWaOu7,Tosca: Io tenni la promessa,1.0,0.0,"['Giacomo Puccini', 'Arturo Basile', 'Orchestr...","['0OzxPXyowUEQ532c9AmHUR', '1J0Clrl4D0cAbGmRKn...",0.260,0.0220,-23.756,Major,0.0392,0.87100,0.073700,0.0879,0.03880,48.591,3,"['Giacomo Puccini', 'Arturo Basile', 'Orchestr...",[],Orchestra Sinfonica,Orchestra Sinfonica|Tosca: Io tenni la promessa,426.400,1956,D,[],Major|D
3,4tQjEbWhRhSmhumjthFsH7,Baby Get Lost,1.0,0.0,"['Dinah Washington', 'Teddy Stewart Orchestra']","['32LHRiof0sa4taYew9i3Fa', '7eXSIjGrfSvF1xDDIJ...",0.527,0.2370,-8.872,Major,0.0597,0.95700,0.000614,0.0612,0.47200,109.626,3,"['Dinah Washington', 'Teddy Stewart Orchestra']",[],Teddy Stewart Orchestra,Teddy Stewart Orchestra|Baby Get Lost,173.067,1957,Eb,[],Major|Eb
4,4dK5XeXIo75DtqDsg6dMah,Down To The River To Pray - Live,4.0,0.0,"['Charlotte Ritchie', 'Sonya Isaacs Yeary', 'J...","['7yK16X5NeP0zuX9qXgcD12', '0YiDah0IN9OfuI9dzd...",0.437,0.4210,-9.466,Major,0.0329,0.86000,0.000000,0.6930,0.53900,134.726,3,"['Charlotte Ritchie', 'Sonya Isaacs Yeary', 'J...",[],Karen Peck,Karen Peck|Down To The River To Pray - Live,197.520,2011,E,[],Major|E
5,4dK5XeXIo75DtqDsg6dMah,Down To The River To Pray - Live,4.0,0.0,"['Charlotte Ritchie', 'Sonya Isaacs Yeary', 'J...","['7yK16X5NeP0zuX9qXgcD12', '0YiDah0IN9OfuI9dzd...",0.437,0.4210,-9.466,Major,0.0329,0.86000,0.000000,0.6930,0.53900,134.726,3,"['Charlotte Ritchie', 'Sonya Isaacs Yeary', 'J...",[],Stephen Hill,Stephen Hill|Down To The River To Pray - Live,197.520,2011,E,[],Major|E
6,3qWI4prgSotUOGR4BfOreW,Clutch Baliye,5.0,0.0,"['Sultaan', 'Gagan']","['7frYUe4C7A42uZqCzD34Y4', '3kfNgIiPN2rdxVEQg6...",0.693,0.5690,-9.153,Major,0.2790,0.00490,0.000000,0.0789,0.42900,179.757,4,"['Sultaan', 'Gagan']",[],Sultaan,Sultaan|Clutch Baliye,152.687,2019,C,"['desi pop', 'punjabi hip hop', 'punjabi pop']",Major|C
7,3hxorGSxTx5s9CcZnOuE04,Madantin (Remix),1.0,0.0,"['Jaywon', 'May D', 'Phyno', 'Olamide']","['5alyYFXdTtyHueFr7XQnGq', '7ahfCquNK3ReYc2UHh...",0.726,0.9410,-2.581,Minor,0.2990,0.37400,0.000025,0.1110,0.74500,124.116,4,"['Jaywon', 'May D', 'Phyno', 'Olamide']",[],Phyno,Phyno|Madantin (Remix),270.811,2014,G,"['afro dancehall', 'afropop', 'azontobeats', '...",Minor|G
8,0zsA45R0SQPfqC5TyDOqY8,Neeyum Naanum Anbe,6.0,0.0,"['Hiphop Tamizha', 'Raghu Dixit', 'Sathyapraka...","['7zFBW2JxM4bgTTKxCRcS8Q', '72578usTM6Cj5qWsi4...",0.467,0.8260,-3.885,Minor,0.0490,0.29800,0.000000,0.2040,0.41700,160.025,4,"['Hiphop Tamizha', 'Raghu Dixit', 'Sathyapraka...",[],Raghu Dixit,Raghu Dixit|Neeyum Naanum Anbe,285.250,2018,E,"['filmi', 'indian folk', 'indian rock', 'kanna...",Minor|E
9,0MLZAgKQKHbPsJ12qHS860,Haan Main Galat,7.0,0.0,"['Pritam', 'Arijit Singh', 'Shashwat Singh']","['1wRPtKGflJrBx9BmLsSwlU', '4YRxDV8wJFPHPTeXep...",0.683,0.7370,-5.660,Major,0.0502,0.12500,0.010500,0.1030,0.91900,117.996,4,"['Pritam', 'Arijit Singh', 'Shashwat Singh']",[],Shashwat Singh,Shashwat Singh|Haan Main Galat,218.644,2020,C,['indian fusion'],Major|C


In [8]:
df_tracks['genres_upd_v1'] = df_tracks['genre_list'].apply(lambda x: re.findall(r"'([^']*)'", x))
df_tracks['genres_upd_v2'] = df_tracks['genre_list'].apply(lambda x: re.findall('\"(.*?)\"',x))
df_tracks['genres_upd'] = np.where(df_tracks['genres_upd_v1'].apply(lambda x: not x), df_tracks['genres_upd_v2'], df_tracks['genres_upd_v1'] )

In [9]:
to_be_scaled = df_tracks.dtypes[df_tracks.dtypes == 'float64'].index.values

In [10]:
df_tracks['genres_upd'][95]

['cuarteto', 'musica blumenauense']

In [11]:
testing = df_tracks['genres_upd'].apply(lambda x: " ".join(x))
testing.head(100)

0                                                      
1                                                      
2                                                      
3                                                      
4                                                      
5                                                      
6                  desi pop punjabi hip hop punjabi pop
7     afro dancehall afropop azontobeats nigerian hi...
8             filmi indian folk indian rock kannada pop
9                                         indian fusion
10    desi pop hindi indie indian indie indian rock ...
11                                                     
12                                          indian folk
13                                      khaliji sheilat
14                            desi pop modern bollywood
15                                  environmental sleep
16                               desi hip hop tamil pop
17    opm pinoy hip hop pinoy r&b pinoy trap tag

In [12]:
complete_feature_set = create_feature_set(df_tracks, to_be_scaled)

In [13]:
complete_feature_set.head()

genre|150  genre|21st  genre|432hz  genre|48g  genre|a3  genre|abc  \
0        0.0         0.0          0.0        0.0       0.0        0.0   
1        0.0         0.0          0.0        0.0       0.0        0.0   
2        0.0         0.0          0.0        0.0       0.0        0.0   
3        0.0         0.0          0.0        0.0       0.0        0.0   
4        0.0         0.0          0.0        0.0       0.0        0.0   

   genre|abstract  genre|acadienne  genre|accordeon  genre|accordion  \
0             0.0              0.0              0.0              0.0   
1             0.0              0.0              0.0              0.0   
2             0.0              0.0              0.0              0.0   
3             0.0              0.0              0.0              0.0   
4             0.0              0.0              0.0              0.0   

   genre|aceh  genre|acid  genre|acidcore  genre|acousmatic  genre|acoustic  \
0         0.0         0.0             0.0               0.0             0.0   
1         0.0         0.0             0.0               0.0             0.0   
2         0.0         0.0             0.0               0.0             0.0   
3         0.0         0.0             0.0               0.0             0.0   
4         0.0         0.0             0.0               0.0             0.0   

   genre|action  genre|adoracao  genre|adoracion  genre|adult  \
0           0.0             0.0              0.0          0.0   
1           0.0             0.0              0.0          0.0   
2           0.0             0.0              0.0          0.0   
3           0.0             0.0              0.0          0.0   
4           0.0             0.0              0.0          0.0   

   genre|adventista  genre|advocacy  genre|afghan  genre|africaine  \
0               0.0             0.0           0.0              0.0   
1               0.0             0.0           0.0              0.0   
2               0.0             0.0           0.0              0.0   
3               0.0             0.0           0.0              0.0   
4               0.0             0.0           0.0              0.0   

   genre|african  genre|afrikaans  genre|afro  genre|afrobeat  \
0            0.0              0.0         0.0             0.0   
1            0.0              0.0         0.0             0.0   
2            0.0              0.0         0.0             0.0   
3            0.0              0.0         0.0             0.0   
4            0.0              0.0         0.0             0.0   

   genre|afrocolombiano  genre|afrofuturism  genre|afrofuturismo  \
0                   0.0                 0.0                  0.0   
1                   0.0                 0.0                  0.0   
2                   0.0                 0.0                  0.0   
3                   0.0                 0.0                  0.0   
4                   0.0                 0.0                  0.0   

   genre|afroperuana  genre|afropop  genre|afroswing  genre|age  genre|aggro  \
0                0.0            0.0              0.0        0.0          0.0   
1                0.0            0.0              0.0        0.0          0.0   
2                0.0            0.0              0.0        0.0          0.0   
3                0.0            0.0              0.0        0.0          0.0   
4                0.0            0.0              0.0        0.0          0.0   

   genre|aggrotech  genre|ahead  genre|ai  genre|aikatsu  genre|ainu  \
0              0.0          0.0       0.0            0.0         0.0   
1              0.0          0.0       0.0            0.0         0.0   
2              0.0          0.0       0.0            0.0         0.0   
3              0.0          0.0       0.0            0.0         0.0   
4              0.0          0.0       0.0            0.0         0.0   

   genre|alabama  genre|alagoana  genre|alagoano  genre|alaska  \
0            0.0             0.0             0.0           0.0  

In [14]:
complete_feature_set.shape

(652305, 2490)

To run the TF-IDF vectorizer I had to export the notebook to my own computer and run it through Jupyter, since my home computer was far more powerful than the free cloud machine I had access too on Deepnote. To make the recommendation system clearer, I'll branch out into a new notebook, on both platforms, and simply end here by exporting the data as a `csv`.

In [15]:
complete_feature_set.to_csv("complete_feature_set.csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6db3d211-80b9-4bca-86b0-008e23f55791' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>